In [4]:
from dotenv import load_dotenv

from pinecone import Pinecone, ServerlessSpec
load_dotenv()

c:\Users\ridae\miniconda3\envs\rag\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [9]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(

name="rag", dimension=384, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")

)

In [12]:
import os
import requests
import json

# Load the JSON data from the reviews.json file
with open("reviews.json", "r") as file:
    data = json.load(file)

# Extract the reviews list from the data
reviews = [review['review'] for review in data['reviews']]

# Hugging Face API details
model_id = "sentence-transformers/all-MiniLM-L6-v2"
hf_token = os.getenv("HUGGINGFACE_API_KEY")

api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}

# Get embeddings for the reviews
response = requests.post(api_url, headers=headers, json={"inputs": reviews, "options": {"wait_for_model": True}})

# Extract the embeddings from the response
embeddings = response.json()

# Prepare the data for Pinecone (assumes embeddings is a list of lists)
processed_data = [
    (f"review-{i}", embedding) for i, embedding in enumerate(embeddings)
]

# Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Create a Pinecone index
pc.create_index(
    name="rag", 
    dimension=len(embeddings[0]),  # Assumes all embeddings have the same dimension
    metric="cosine", 
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

# Connect to the index
index = pc.Index('rag')

# Upsert the processed data into Pinecone
index.upsert(
    vectors=processed_data,
    namespace="nsl"
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '2789ed5f6979e22f4b50c711f69ad1c8', 'Date': 'Sat, 24 Aug 2024 19:34:01 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}
